In [25]:
import pandas as pd
import numpy as np
import os

prepare_RSgroups = False
dataset_name = 'house_prices'

DIR_TO_SAVE_RES = './data/'+dataset_name+'/first_test/'

In [2]:
if prepare_RSgroups:
    for i in range(1,11):
        part_1 = pd.read_csv(DIR_TO_SAVE_RES+\
                             "res_rand_group1_house_prices_{}.csv".format(i))
        part_2 = pd.read_csv(DIR_TO_SAVE_RES+\
                             "res_rand_group2_house_prices_{}.csv".format(i))
        part_3 = pd.read_csv(DIR_TO_SAVE_RES+\
                             "res_rand_group3_house_prices_{}.csv".format(i))
        full = pd.concat([part_1,part_2,part_3], sort = False)
        full['run_number']=i
        print(full.shape)
        full.to_csv(DIR_TO_SAVE_RES+"res_rand_groups_{}_{}.csv".format(dataset_name, i), index=False)

(729, 21)
(729, 21)
(729, 21)
(729, 21)
(729, 21)
(729, 21)
(729, 21)
(729, 21)
(729, 21)
(729, 21)


In [3]:
RS_time = np.array([27.66507022380829, 28.061819970607758, 27.44816317160924,
           27.645976877212526, 26.519640775521598, 27.433769524097443,
           28.03506429195404, 27.815785245100656])
print(np.mean(RS_time))
print(np.std(RS_time))

27.578161259988946
0.4568256649999829


In [4]:
group_1_time = np.array([13.971220242977143, 13.86060604651769, 13.980982772509257,
                        13.661206158002217, 13.581735169887542])
group_2_time = np.array([9.980452716350555, 9.539682837327321, 10.220448072751363,
                        9.545726998647053, 9.52247104247411])
group_3_time = np.array([10.060758105913798, 9.634749031066894, 10.268958103656768,
                        9.60460832118988, 9.623231919606527])

sum_arr=np.sum([group_1_time, group_2_time, group_3_time], axis = 0)


for arr in [group_1_time, group_2_time, group_3_time, sum_arr]:
    print(np.mean(arr))
    print(np.std(arr))
    print("=====")

13.81115007797877
0.16249669157903252
=====
9.76175633351008
0.2868689530953342
=====
9.838461096286775
0.27468271087739105
=====
33.41136750777562
0.700282576201665
=====


In [2]:
def group_best_params_with_stop(df, stop_iter_num, left_iter_num, right_iter_num):
    best_params = {}
    stop_iter_nums = {}
    best_score = {}

    tmp_df = df.loc[(df['iter_num'] >= left_iter_num) & (df['iter_num'] <= right_iter_num)].copy() #take group
    for i in tmp_df.run_number.unique(): 
      tmp_df_i = tmp_df.loc[tmp_df.run_number==i] #take run
      #cut iterations:
      tmp_df_i_copy = tmp_df_i.copy()
      tmp_df_i_copy.loc[:, "mean_test_score"] = tmp_df_i_copy.loc[
          :, "mean_test_score"].cummax()      

      try:
          curr_max=tmp_df_i_copy.groupby("mean_test_score").count().reset_index().sort_values(by = "mean_test_score")
          max_score = curr_max.loc[curr_max["mean_test_score"]>=stop_iter_num].index[0] #get first interval with iter_amount more than stop_iter_num
      except:
          max_score = tmp_df_i_copy.mean_test_score.max()

      tmp_stop = tmp_df_i.loc[tmp_df_i["mean_test_score"]==max_score].sort_values(by="iter_num").iloc[0]#get first element with best_score
    
      best_params[i] = tmp_stop['params']
      #best_score[i] = tmp_stop['mean_test_score']

      last_iter_num = tmp_stop['iter_num'] + stop_iter_num - 1
      if last_iter_num > right_iter_num:
          last_iter_num = right_iter_num

      stop_iter_nums[i] = last_iter_num
    return best_params, stop_iter_nums#, best_score       
   

In [26]:
group_df = pd.DataFrame([])
for i in range(1,11):
    df = pd.read_csv(DIR_TO_SAVE_RES+"res_rand_groups_{}_{}.csv".format(dataset_name, i))    
    df['iter_num'] = range(1, df.shape[0]+1)
    group_df = pd.concat([group_df, df], sort = False)

In [27]:
stop_iter_num = 25
best_params_first_group, first_stop_iter_nums = group_best_params_with_stop(
    group_df, stop_iter_num = stop_iter_num,
    left_iter_num=1, right_iter_num=243)

In [28]:
best_params_first_group

{1: "{'max_depth': 3, 'min_child_weight': 3}",
 2: "{'max_depth': 3, 'min_child_weight': 3}",
 3: "{'max_depth': 3, 'min_child_weight': 3}",
 4: "{'max_depth': 3, 'min_child_weight': 3}",
 5: "{'max_depth': 3, 'min_child_weight': 3}",
 6: "{'max_depth': 3, 'min_child_weight': 3}",
 7: "{'max_depth': 3, 'min_child_weight': 5}",
 8: "{'max_depth': 3, 'min_child_weight': 3}",
 9: "{'max_depth': 3, 'min_child_weight': 3}",
 10: "{'max_depth': 3, 'min_child_weight': 4}"}

In [29]:
first_stop_iter_nums

{1: 178, 2: 136, 3: 128, 4: 223, 5: 84, 6: 214, 7: 57, 8: 109, 9: 235, 10: 229}

In [3]:
import os 

In [17]:
os.listdir("./data/tmp")

['GR_S_2_group_with_10_6_house_prices.csv',
 'GR_S_2_group_with_100_9_house_prices.csv',
 'GR_S_2_group_with_25_8_house_prices.csv']

In [6]:
group_df = pd.read_csv("./data/tmp/GR_S_2_group_with_10_6_house_prices.csv")
stop_iter_num = 10

first_group_stop_iters = 199
left_iter_num = first_group_stop_iters+1
right_iter_num =left_iter_num + int(round((729 - first_group_stop_iters)/2))

best_params_second_group, second_stop_iter_nums = group_best_params_with_stop(
    group_df, stop_iter_num = stop_iter_num,
    left_iter_num=left_iter_num, right_iter_num=right_iter_num)

In [7]:
best_params_second_group

{6: "{'alpha': 0.664132198774426, 'lambda': 0.2752314671549202}"}

In [8]:
second_stop_iter_nums

{6: 209}

In [30]:
group_df = pd.read_csv("./data/tmp/GR_S_2_group_with_25_8_house_prices.csv")
# {'max_depth': 3, 'min_child_weight': 3}

stop_iter_num = 100

first_group_stop_iters = 109
left_iter_num = first_group_stop_iters+1
right_iter_num =left_iter_num + int(round((729 - first_group_stop_iters)/2))

best_params_second_group, second_stop_iter_nums = group_best_params_with_stop(
    group_df, stop_iter_num = stop_iter_num,
    left_iter_num=left_iter_num, right_iter_num=right_iter_num)

In [31]:
best_params_second_group

{8: "{'alpha': 0.5391121857294692, 'lambda': 0.7030791263648584}"}

In [32]:
second_stop_iter_nums

{8: 209}